<a href="https://colab.research.google.com/github/SourCherries/VAE-GAN-CelebA/blob/master/learn_tf_conv2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# https://towardsdatascience.com/understanding-convolutions-by-hand-vs-tensorflow-8e64053f673e
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


In [3]:
def get_filter():
    Wk=3
    Hk=3
    Dk=3
    w=np.zeros(shape=(Wk,Hk,Dk)).astype(int) #initialize 3x3x3 kernel
    w[:,:,0]=np.array(([1,0,-1],[0,0,0],[-1,0,1]),dtype='int')
    w[:,:,1]=np.array(([0,-1,0],[-1,4,-1],[0,-1,0]),dtype='int')
    w[:,:,2]=np.array(([-1,-1,-1],[-1,8,-1],[-1,-1,-1]),dtype='int')
    return w

In [4]:
W=5 #width of input array
H=5 #height of input array
D=3 #depth of input array
# random x volume for example
np.random.seed(1) #for reproducibility
x=np.random.rand(W,H,D)*4 #up to 4 random numbers 
x=x.astype(int)

In [5]:
def conv2d_pad_str(x,w0,S=2,b0=np.ones(shape=(1))):
    if len(x.shape)>2:
        xc=x.shape[2]
    else:
        xc=1
    xl=x.shape[1] #input volume length
    xw=x.shape[0] #input volume width
    fl=w0.shape[1] #filter length
    fw=w0.shape[0] #filter width
    ol=(xl-fl)//S+1 #output volume length
    ow=(xw-fw)//S+1 #output volume width
    if len(x.shape)>2: #
        output=np.zeros(shape=(ol,ow,1))
        output[:,:,0]=output[:,:,0]+b0
    else:
        output=np.zeros(shape=(ol,ow))
        output[:,:]=output[:,:]+b0

    for c in range(0,xc):
        cj=0
        for j in range(0,xl+S,S):
            ci=0
            for i in range(0,xw+S,S):
                if i>xw-fw: #reached the end of the row, breaks to next row
                    break
                if j>xl-fl: #reached the end of the column, breaks to next column
                    break
                if len(x.shape)>2:
                    output_ji=np.sum(x[:,:,c][j:j+fl,i:i+fw]*w0[:,:,c])
                else:
                    output_ji=np.sum(x[:,:][j:j+fl,i:i+fl]*w0[:,:])
                if len(x.shape)>2:
                    output[cj,ci,0]+=output_ji
                else:
                    output[cj,ci]+=output_ji
                ci+=1
            cj+=1
    return output

In [6]:
def tensorflow_conv(x,w,bias,S):
  global tf_output
  x=np.asarray(x).astype(np.float32)
  x_tf=tf.constant(np.expand_dims(x,axis=0),dtype=tf.float32)
  f_tf=tf.constant(np.expand_dims(w,axis=0),dtype=tf.float32)
  x_tf=tf.reshape(x_tf,shape=(1,x.shape[0],x.shape[1],x.shape[2]))
  f_tf=tf.reshape(f_tf,shape=(w.shape[0],w.shape[1],w.shape[2],1))
  b_tf=tf.constant(bias,dtype=tf.float32)
  conv_tf=tf.nn.conv2d(x_tf,f_tf,strides=[1,S,S,1],padding='VALID')
  conv_tf=tf.nn.bias_add(conv_tf,b_tf)[0,:,:,0]
  tf_output=conv_tf.numpy()
  return tf_output

In [7]:
w = get_filter()
bias = np.ones(shape=(1))

In [8]:
print(w.shape)
print(w[:,:,0])


(3, 3, 3)
[[ 1  0 -1]
 [ 0  0  0]
 [-1  0  1]]


In [9]:
# tf_output = tensorflow_conv(x, w, bias, S=2)

AttributeError: ignored

In [9]:
S = 1
x=np.array(x).astype(np.float32)
x_tf=tf.constant(np.expand_dims(x,axis=0),dtype=tf.float32)
f_tf=tf.constant(np.expand_dims(w,axis=0),dtype=tf.float32)
x_tf=tf.reshape(x_tf,shape=(1,x.shape[0],x.shape[1],x.shape[2]))
f_tf=tf.reshape(f_tf,shape=(w.shape[0],w.shape[1],w.shape[2],1))
b_tf=tf.constant(bias,dtype=tf.float32)
conv_tf=tf.nn.conv2d(x_tf,f_tf,strides=[1,S,S,1],padding='VALID')
conv_tf=tf.nn.bias_add(conv_tf,b_tf)[0,:,:,0]

In [10]:
tf_output=conv_tf.numpy()

AttributeError: ignored

In [ ]:
print(conv_tf)
print(conv_tf.shape)

In [15]:
# https://stackoverflow.com/questions/34097281/convert-a-tensor-to-numpy-array-in-tensorflow

print(type(tf.Session().run( conv_tf )))

<class 'numpy.ndarray'>


In [18]:
cout = tf.Session().run( conv_tf )
print(cout)

[[ 15.   5.  -3.]
 [  1.  -7.  11.]
 [-17.  -6.  -1.]]
